In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from pos_processing import graph_results, fem_analysis
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 2: Coaxial Cable with longitudinal symmetry

Considere a seção reta transversal de um cabo coaxial singelo apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_longitudinal.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

# 6-noded Quadratic Triangular Elements, $P_2$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 2)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0, 'name': 'outer_sheath'}]

MATERIAL = [{'tag': 201, 'name': 'core', 'relative_magnetic_permeability': 1},
            {'tag': 202, 'name': 'dielectric', 'relative_magnetic_permeability': 1},
            {'tag': 203, 'name': 'sheath', 'relative_magnetic_permeability': 1}]

# Criar geometria coaxial
create_domain.coaxial_longitudinal(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-3, view_mesh=False)

# Create mesh from file geometry
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coax_long')

Gerando malha com elementos triangulares de ordem P2 ...
Malha salva em pre_processing/mesh/coax_long_domain_Triangle2.msh
Model coax_long_domain_Triangle2 (2D)
Info     : 9 geometric entities
Info     : 4 Physical Groups
Info     : 6131 nodes in total
Info     : 3002 2-D elements in total


### Mesh Data Dictionaries

In [3]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 302,
  'conn_list': [883, 913, 824, 948, 949, 950],
  'material': {'tag': 201,
   'name': 'core',
   'relative_magnetic_permeability': 1}}]

In [4]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.01, 0.0),
  'bc': {'type': 'Free', 'value': None}}]

### ``Nodes`` and ``conn`` list

In [5]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

### Figure 1: Coaxial meshed domain 

In [6]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coax_long', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_magnetostatics\pre_processing\pictures\coax_long_meshed_domain_Triangle2.svg


<figure>
    <img src="pre_processing/pictures/coax_long_meshed_domain_Triangle2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

### Assembly Matrices
#### `assembly.global_nodes_coordinates()` 

In [7]:
ai, xi, yi = matrices_assembly.global_nodes_coordinates(0, mesh_data)
print(f"'a_1' Global coordinates: {ai}")
print(f"'x_1' x-coordinates: {xi}")
print(f"'y_1' y-coordinates: {yi}")

'a_1' Global coordinates: [(-0.006838045274302086, 0.004491270025444997), (-0.007942853716170466, 0.00439813878217982), (-0.007329955473093625, 0.003660359087928205), (-0.007390449495236277, 0.004444704403812409), (-0.007636404594632046, 0.004029248935054013), (-0.007084000373697856, 0.004075814556686601)]
'x_1' x-coordinates: [-0.006838045274302086, -0.007942853716170466, -0.007329955473093625, -0.007390449495236277, -0.007636404594632046, -0.007084000373697856]
'y_1' y-coordinates: [0.004491270025444997, 0.00439813878217982, 0.003660359087928205, 0.004444704403812409, 0.004029248935054013, 0.004075814556686601]


#### `map_to_physical_coordinates()`

In [8]:
xi_master = (1, 0)
xg, yg = matrices_assembly.isomapping_to_global_coordinates(ai, xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg}, {yg})")

'e_1' Master coordinates: (1, 0) --> Global coordinates: (-0.007942853716170466, 0.00439813878217982)


#### `apply_physics()`

In [9]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 302,
  'conn_list': [883, 913, 824, 948, 949, 950],
  'material': {'tag': 201,
   'name': 'core',
   'relative_magnetic_permeability': 1},
  'source': {'type': 'uniform_current_density_Jz', 'value': 3183.098861837907},
  'stiffness_a_value': 795774.7150262763}]

#### Material Proprieties, $k_a$

In [10]:
Ka = [cell['stiffness_a_value'] for cell in mesh_data['conn_data']]
Ma = [cell['mass_a_value'] for cell in mesh_data['conn_data'] if 'mass_a_value' in cell.keys()]
print("Stiffness Material values: ", Ka[:3])
print("Mass Material values: ", Ma[:3])

Stiffness Material values:  [795774.7150262763, 795774.7150262763, 795774.7150262763]
Mass Material values:  []


#### Source: Current density, $J_z$

In [11]:
Jz = [element['source']['value'] for element in mesh_data['conn_data']]
print("Source current density vector Jz: \n", Jz[:1])

Source current density vector Jz: 
 [3183.098861837907]


#### Jacobian Matrix Transform

In [12]:
Je = matrices_assembly.jacobian(e=0, mesh_data=mesh_data, element_type=FINITE_ELEMENT, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_0:\n", Je)
print("Determinant of the Jacobian matrix for element e_0:", Jdet)

Jacobian matrix for element e_0:
 [[-1.10480844e-03 -9.31312433e-05]
 [-4.91910199e-04 -8.30910938e-04]]
Determinant of the Jacobian matrix for element e_0: 8.721852098210483e-07


#### Local Elements $e_1$

In [13]:
#Ae, fe, areae = matrices_assembly.linear_local_matrices(e=0, mesh_data=mesh_data)
Ae, fe, Me, area_e = matrices_assembly.higher_order_local_matrices(0, mesh_data, FINITE_ELEMENT)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

Local stiffness matrix for element e_1:
 [[ 4.19683451e+05  4.73741726e+04  9.25203102e+04 -1.89496689e+05
  -2.75763028e-03 -3.70081242e+05]
 [ 4.73741726e+04  4.25351909e+05  9.44097981e+04 -1.89496691e+05
  -3.77639191e+05  1.82582958e-03]
 [ 9.25203102e+04  9.44097981e+04  5.60790322e+05  2.70104104e-03
  -3.77639191e+05 -3.70081242e+05]
 [-1.89496689e+05 -1.89496691e+05  2.70104104e-03  1.87443424e+06
  -7.40162478e+05 -7.55278389e+05]
 [-2.75763028e-03 -3.77639191e+05 -3.77639191e+05 -7.40162478e+05
   1.87443424e+06 -3.78993376e+05]
 [-3.70081242e+05  1.82582958e-03 -3.70081242e+05 -7.55278389e+05
  -3.78993376e+05  1.87443425e+06]]
Local load vector for element e_1:
 [[ 1.11183903e-12]
 [-1.88591225e-14]
 [-1.88582331e-14]
 [ 4.62708621e-04]
 [ 4.62708617e-04]
 [ 4.62708621e-04]]
Area of element e_1:  4.3609260054959637e-07


#### Global “stiffness” matrix, $A_g$

In [14]:
Ag, fg, Mg = matrices_assembly.get_global_matrix(mesh_data, FINITE_ELEMENT)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (6131, 6131)


### Imposition of Boundary Conditions

In [15]:
dirichlet_data = {node['TagID']: node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet'}
Nu = len(mesh_data['nodes_data'])
Nd = len(dirichlet_data)
print(f"The entire domain has {Nu} nodes: {Nu - Nd} free nodes and {Nd} Dirichlet nodes.")

The entire domain has 6131 nodes: 5879 free nodes and 252 Dirichlet nodes.


### Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [ ]:
global_to_reduced = {node: i for i, node in enumerate(
        idx_n for idx_n in range(1, len(nodes)+1) if idx_n not in dirichlet_data)}

## Processor Module
### Reduced Global matrix $A_{gr}$ with boundary conditions

In [17]:
Agr, fgr, Mgr = boundary_conditions.get_reduced_global_matrices(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (5879, 5879)


### Global Potential Vector

In [18]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("Reduced solution vector ur shape: ", ur.shape)
print("Global potential vector V shape: ", potential_u.shape)

Reduced solution vector ur shape:  (5879,)
Global potential vector V shape:  (6131,)


### Solution at each element

In [19]:
u_list = []
for e in range(len(conn)):
    uh_values = [potential_u[id - 1] for id in conn[e]]
    u_list.append(uh_values)
print("The solution u for Node Element is: \n", np.array(u_list[:3]))

The solution u for Node Element is: 
 [[1.58003601e-07 1.42501066e-07 1.57807490e-07 1.50559787e-07
  1.50384374e-07 1.58138624e-07]
 [1.55959125e-07 1.59632064e-07 1.42255468e-07 1.58047641e-07
  1.51200447e-07 1.49506237e-07]
 [1.42255468e-07 1.38793745e-07 1.55959125e-07 1.40755282e-07
  1.47617099e-07 1.49506237e-07]]


## Post-Processor

In [20]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='coax_long')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_magnetostatics\pos_processing\pictures\coax_long_domain_solution_Triangle2.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh

<figure>
    <img src="pos_processing/pictures/coax_long_domain_solution_Triangle2.svg" alt="Fig.2" style="width:80%;" />
    <figcaption>Figure 2: Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

### Magnetic Energy Stored

In [21]:
analytical_solution = lambda x, y: ps.potential_and_gradient(x, y)
norm_l2, norm_energy, total_energy = fem_analysis.errors_and_energy(
            mesh_data, FINITE_ELEMENT, potential_u, analytical_solution)
print(f"Total magnetic energy stored: {total_energy:.5} J.")

Total magnetic energy stored: 8.5076e-08 J.


### `magnetic_field(mesh_data, element_type, uh)`

In [22]:
B_field = fem_analysis.magnetic_field_density(mesh_data, FINITE_ELEMENT, potential_u)
graph_results.magnetic_field_distribution(mesh_data, B_field, FINITE_ELEMENT)
graph_results.radial_profile(mesh_data, B_field, FINITE_ELEMENT)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_magnetostatics\pos_processing\pictures\b_field_Triangle2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_magnetostatics\pos_processing\pictures\radial_b_profile_Triangle2.svg


### Figure 3: Magnetic field density distribution

<figure>
    <img src="pos_processing/pictures/b_field_Triangle2.svg" alt="Fig.3" style="width:90%;" />
    <figcaption>Figure 3: Magnetic field density distribution.</figcaption>
</figure>

### Figure 4: Magnetic density field radial profile

<figure>
    <img src="pos_processing/pictures/radial_b_profile_Triangle2.svg" alt="Fig.4" style="width:90%;" />
    <figcaption>Figure 4: Magnetic field density radial profile.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.